In [3]:
from langchain.sql_database import SQLDatabase
from langchain.chat_models import ChatOpenAI
from langchain import SQLDatabaseChain
from urllib.parse import quote  
from sqlalchemy import create_engine, select, Table, MetaData, Column, String
from dotenv import load_dotenv
import os

load_dotenv()

True

# Gerando consulta SQL equivalente a da consulta em linguagem natural através do ChatGPT com auxílio do Langchain e um prompt criado manualmente


O langchain automatiza o processo de criação de um prompt para um esquema de banco de dados. Para responder consultas sobre um banco de dados SQL, o langchain usa o SQLDatabaseChain, cuja tarefa principal é fornecer à GPT informações de bancos de dados personalizados. Além disso, pode-se personalizar o prompt usado e adicionar mais informações ao GPT


### Conectando ao Oracle Database

https://analytix.nl/post/querying-complex-database-schemas-with-gpt-and-langchain/
https://python.langchain.com/docs/modules/chains/popular/sqlite
https://python.langchain.com/docs/modules/agents/toolkits/sql_database


In [17]:
username = str(os.getenv('DB_USER_NAME'))
passwd = str(os.getenv('DB_PASS'))
hostname = str(os.getenv('DB_HOST'))
port = str(os.getenv('DB_PORT'))
database = str(os.getenv('DB_NAME'))
sqldriver = str(os.getenv('SQL_DRIVER'))
servicename = str(os.getenv('SERVICE_NAME'))

# dsn = """(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST={hostname})
#     (PORT={port}))(CONNECT_DATA=(SERVER=DEDICATED)(SERVICE_NAME={servicename})))"""
    
import cx_Oracle
dsnStr = cx_Oracle.makedsn(host = hostname, port = port, service_name = servicename).replace("SID", "SERVICE_NAME")
    
uri = "{}://{}:{}@{}".format(sqldriver, username, quote(passwd), dsnStr)
print(uri)
db = SQLDatabase.from_uri(uri)

oracle+oracledb://mondial_sql:mondial_sql@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=beatriz.tecgraf.puc-rio.br)(PORT=1521))(CONNECT_DATA=(SERVICE_NAME=orcl.tecgraf.puc-rio.br)))


Testando a conexão

In [18]:
db.get_table_names()

['airport',
 'borders',
 'city',
 'citylocalname',
 'cityothername',
 'citypops',
 'continent',
 'country',
 'countrylocalname',
 'countryothername',
 'countrypops',
 'desert',
 'dr$i_mondial_tmdc_test$i',
 'dr$i_mondial_tmdc_test$k',
 'dr$i_mondial_tmdc_test$n',
 'dr$i_mondial_tmdc_test$r',
 'dr$i_mondial_tmdc_test$u',
 'dr$i_mondial_tmdp_test$i',
 'dr$i_mondial_tmdp_test$k',
 'dr$i_mondial_tmdp_test$n',
 'dr$i_mondial_tmdp_test$r',
 'dr$i_mondial_tmdp_test$u',
 'dr$i_mondial_tpv_test$i',
 'dr$i_mondial_tpv_test$k',
 'dr$i_mondial_tpv_test$n',
 'dr$i_mondial_tpv_test$r',
 'dr$i_mondial_tpv_test$u',
 'dr$idx_ctx_00d49b9c00d8b7c23$i',
 'dr$idx_ctx_00d49b9c00d8b7c23$k',
 'dr$idx_ctx_00d49b9c00d8b7c23$n',
 'dr$idx_ctx_00d49b9c00d8b7c23$r',
 'dr$idx_ctx_00d49b9c00d8b7c23$u',
 'dr$idx_ctx_011be7ab6d6a4a465$i',
 'dr$idx_ctx_011be7ab6d6a4a465$k',
 'dr$idx_ctx_011be7ab6d6a4a465$n',
 'dr$idx_ctx_011be7ab6d6a4a465$r',
 'dr$idx_ctx_011be7ab6d6a4a465$u',
 'dr$idx_ctx_0dea24ec14ddda69e$i',
 'dr$idx

Após importar e rodar localmente, gerou erros,
- o SQLAlchemy não sabia que tipo era GEOCOORD

- o número de tokens enviados ao modelo referente a informações do banco de dados foi maior que o limite máximo de tokens(4097).


Então a próxima abordagem foi diminuir o tamanho da base de dados, adicionando 
2 linhas de exemplos para cada tabela

In [19]:
include_tables = ["airport", "borders", "city", "citylocalname", "cityothername", "citypops", "continent", "country", 
                  "countrylocalname", "countryothername", "countrypops", "economy", "encompasses", "ethnicgroup", "organization", 
                  "politics", "population", "province", "provincelocalname", "provinceothername", "provpops", "religion"]

db = SQLDatabase.from_uri(uri, include_tables=include_tables, sample_rows_in_table_info=2)

db.get_table_names()

['airport',
 'borders',
 'city',
 'citylocalname',
 'cityothername',
 'citypops',
 'continent',
 'country',
 'countrylocalname',
 'countryothername',
 'countrypops',
 'economy',
 'encompasses',
 'ethnicgroup',
 'organization',
 'politics',
 'population',
 'province',
 'provincelocalname',
 'provinceothername',
 'provpops',
 'religion']

Verificando o prompt que o langchain passa para o GPT

In [20]:
print(db.table_info)
f = open("prompt_personalizado.txt", "w")
f.write(db.table_info)
f.close()



CREATE TABLE airport (
	iatacode VARCHAR(3 CHAR) NOT NULL, 
	name VARCHAR(100 CHAR), 
	country VARCHAR(4 CHAR), 
	city VARCHAR(50 CHAR), 
	province VARCHAR(50 CHAR), 
	island VARCHAR(50 CHAR), 
	latitude NUMBER, 
	longitude NUMBER, 
	elevation NUMBER, 
	gmtoffset NUMBER, 
	CONSTRAINT sys_c00114043 PRIMARY KEY (iatacode), 
	CONSTRAINT airplat CHECK ((Latitude >= -90) AND (Latitude <= 90)), 
	CONSTRAINT airplon CHECK ((Longitude >= -180) AND (Longitude <= 180))
)

/*
2 rows from airport table:
iatacode	name	country	city	province	island	latitude	longitude	elevation	gmtoffset
KMG	Wujiaba	CN	Kunming	Yunnan	None	24.992364	102.743536	1895.0	8.0
TSN	Binhai	CN	Tianjin	Tianjin	None	39.124353	117.346183	3.0	8.0
*/


CREATE TABLE borders (
	country1 VARCHAR(4 CHAR) NOT NULL, 
	country2 VARCHAR(4 CHAR) NOT NULL, 
	length NUMBER, 
	CONSTRAINT borderkey PRIMARY KEY (country1, country2), 
	CONSTRAINT sys_c00113990 CHECK (Length > 0)
)

/*
2 rows from borders table:
country1	country2	length
AL	GR	282.

Customizando um prompt

In [22]:
from langchain.prompts.prompt import PromptTemplate

_DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run,
In the syntax, column and table names must be in uppercase and Don't use quotes for column names.
We use oracle database.

Then, look at the results of the query and return the answer. Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:
{table_info}



Question: {input}"""
PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
)

Criando o modelo

In [23]:
db_chain = SQLDatabaseChain(llm=ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo'), database=db, prompt=PROMPT, verbose=True)

c:\Users\wendyzv\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain\chains\sql_database\base.py:63: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [24]:
def execute_query(query):
    try:
        db_chain.run(query)
    except:
        print("\n Consulta SQL gerada, mas houve erro ao executá-la")

In [25]:
execute_query("Country limit 10")



> Entering new  chain...
Country limit 10
SQLQuery:SELECT * FROM country LIMIT 10
 Consulta SQL gerada, mas houve erro ao executá-la


Sintaticamente está correta, mas deu erro, como o banco é oracle e não usa o LIMIT

In [41]:
execute_query("Country with largest area")



> Entering new  chain...
Country with largest area
SQLQuery:SELECT name
FROM country
WHERE area = (SELECT MAX(area) FROM country)
SQLResult: [('Russia',)]
Answer:Country with largest area: Russia
> Finished chain.


In [42]:
execute_query("What province is the city of Bandar Seri Begawan in?")



> Entering new  chain...
What province is the city of Bandar Seri Begawan in?
SQLQuery:SELECT province
FROM city
WHERE name = 'Bandar Seri Begawan';
 Consulta SQL gerada, mas houve erro ao executá-la


In [30]:
execute_query("What is relationship between India and Mauritius Island?")




> Entering new  chain...
What is relationship between India and Mauritius Island?
SQLQuery:SELECT * FROM borders WHERE (country1 = 'IN' AND country2 = 'MUS') OR (country1 = 'MUS' AND country2 = 'IN')
SQLResult: []
Answer:Final answer here: There is no direct border relationship between India and Mauritius Island.
> Finished chain.


In [32]:
execute_query("What percentage of religious people are hindu in India?")



> Entering new  chain...
What percentage of religious people are hindu in India?
SQLQuery:SELECT Percentage
FROM religion
WHERE country = 'IND' AND name = 'Hindu'
SQLResult: [(Decimal('79.8'),)]
Answer:Final answer here: 79.8%
> Finished chain.


In [31]:
execute_query("Tell me five brazilians cities")



> Entering new  chain...
Tell me five brazilians cities
SQLQuery:SELECT name
FROM city
WHERE country = 'BR'
FETCH FIRST 5 ROWS ONLY;
 Consulta SQL gerada, mas houve erro ao executá-la


In [28]:

execute_query("tell me percentage of religious by religion in india")



> Entering new  chain...
tell me percentage of religious by religion in india
SQLQuery:SELECT name, percentage
FROM religion
WHERE country = 'INDIA'
SQLResult: []
Answer:Final answer here: There are no records in the religion table for the country 'INDIA'.
> Finished chain.


In [34]:
execute_query("tell me total percentage of religious people who are Hindu by continent")



> Entering new  chain...
tell me total percentage of religious people who are Hindu by continent
SQLQuery:SELECT c.name, SUM(r.percentage) AS total_percentage
FROM continent c
JOIN encompasses e ON c.name = e.continent
JOIN country co ON e.country = co.code
JOIN religion r ON co.code = r.country
WHERE r.name = 'Hindu'
GROUP BY c.name
SQLResult: [('Asia', Decimal('261.5')), ('Europe', Decimal('6.6')), ('Australia/Oceania', Decimal('33.4')), ('Africa', Decimal('59.7')), ('South America', Decimal('47.1')), ('North America', Decimal('33.4'))]
Answer:Total percentage of religious people who are Hindu by continent is as follows:
- Asia: 261.5%
- Europe: 6.6%
- Australia/Oceania: 33.4%
- Africa: 59.7%
- South America: 47.1%
- North America: 33.4%
> Finished chain.


In [29]:
#Essa consulta funcionou no software cliente do oracle, mas não funcionou aqui
execute_query("tell me airports with elevation more than 1000")



> Entering new  chain...
tell me airports with elevation more than 1000
SQLQuery:SELECT iatacode, name, elevation
FROM airport
WHERE elevation > 1000
SQLResult: [('KMG', 'Wujiaba', 1895), ('BAV', 'Baotou Airport', 1013), ('LXA', 'Lhasa-Gonggar', 4005), ('KWE', 'Longdongbao', 1139), ('XNN', 'Xining Caojiabao Airport', 2170), ('YUS', 'Yushu Batang', 3963), ('KHG', 'Kashi', 1381), ('AKU', 'Aksu Airport', 1163), ('DAT', 'Datong Airport', 1049), ('BOG', 'Eldorado Intl', 2549), ('AXM', 'El Eden', 1216), ('MZL', 'La Nubia', 2095), ('PEI', 'Matecana', 1346), ('BGA', 'Palonegro', 1188), ('EOH', 'Olaya Herrera', 1506), ('FBM', 'Lubumbashi Intl', 1309), ('SYQ', 'Tobias Bolanos International Airport', 1002), ('GUA', 'La Aurora', 1510), ('TGU', 'Toncontin Intl', 1004), ('SXR', 'Srinagar', 1655), ('UIO', 'Mariscal Sucre Intl', 2813), ('CUE', 'Mariscal Lamar', 2532), ('ASM', 'Asmara Intl', 2336), ('GDQ', 'Gondar', 1994), ('DIR', 'Dire Dawa Intl', 1167), ('ADD', 'Bole Intl', 2334), ('KBL', 'Kabul In

## Conclusões 

Tive problemas com tipos de colunas customizadas do banco, no caso o Geocoord, pois o SqlAlchemy não o reconhece. Pode ser contornado nesse [link](https://docs.sqlalchemy.org/en/20/core/custom_types.html#sqlalchemy.types.UserDefinedType), mas não consegui implementar. 

Então importei o banco para o local e alterei as colunas com tipo Geocoord para Varchar(50) e assim, consegui contrornar. Ao criar o SQLDatabaseChain e executar uma consulta tive erro pois o prompt gerado pelo langchain passava  do limite máximo de tokens. 

Em sua documentação, o langchain oferece a possibilidade de incluir e excluir quais tabelas queremos. Então, retornei e fiz a comunicação com o banco "mondial_sql" do servidor e fiz as consultas. Gerava o SQL, mas dava DatabaseError e não retornava os resultados. E para verificar e validar se o SQL gerado trazia resultados, a consulta gerada pelo GPT era passada diretamente em um software cliente SGBD  para ser executada.

Algumas consultas SQL geradas estavam retornando as colunas com aspas e dava erro na execução da consulta, tanto aqui quanto no software cliente.
```
SELECT "name" 
FROM country 
ORDER BY area DESC 
FETCH FIRST 1 ROWS ONLY;
```
Quando removi elas, a consulta foi executada corretamente no software cliente.

As consultas SQL geradas que não possuiam aspas em algumas das colunas executavam normalmente no software cliente, mas aqui estava dando erro.

Nessa consulta "What is relationship between India and Mauritius Island?" o sql gerado buscou uma relação entre esses dois lugares pela coluna island da tabela airport (na abordagem passando o prompt foi por Borders). Mas não houve resultados.

Interessante, que nessa consulta "tell me total percentage of religious people who are Hindu by continent" trouxe resultados corretos onde envolvia 3 joins e agregação. O danke não respondeu correto para as seguintes palavras chaves:
- "percentage total hindu continent" "total percentage hindu continent"
- "percentage sum hindu continent" "sum total hindu continent"

In [30]:
execute_query("What are the provinces with an area greater than 1000?")



> Entering new  chain...
What are the provinces with an area greater than 1000?
SQLQuery:SELECT name
FROM province
WHERE area > 1000
SQLResult: [('Primorskiy',), ('Khabarovskiy',), ('Amurskaya',), ('Kamchatka',), ('Magadanskaya',), ('Sakhalin',), ('Khanty Mansi ao',), ('Yamalo Nenets ao',), ('Antwerpen',), ('Oost-Vlaanderen',), ('Hainaut',), ('LiÃ¨ge',), ('Limburg',), ('Luxembourg',), ('Namur',), ('West-Vlaanderen',), ('Brabant Wallon',), ('Vlaams-Brabant',), ('Luxembourg',), ('Groningen',), ('Friesland',), ('Drenthe',), ('Overijssel',), ('Flevoland',), ('Gelderland',), ('Utrecht',), ('Noord-Holland',), ('Zuid-Holland',), ('Zeeland',), ('Noord-Brabant',), ('Limburg',), ('Slovenia',), ('Croatia',), ('Federacija Bosne i Hercegovine',), ('Republika Srpska',), ('Bulgaria',), ('Alba',), ('Arad',), ('ArgeÅŸ',), ('BacÄƒu',), ('Bihor',), ('BistriÅ£a-NÄƒsÄƒud',), ('BotoÅŸani',), ('BrÄƒila',), ('BraÅŸov',), ('BuzÄƒu',), ('CÄƒlÄƒraÅŸi',), ('CaraÅŸ-Severin',), ('Cluj',), ('ConstanÅ£a',), ('Covas

In [40]:
execute_query("What are the provinces with an area more than 1000?")



> Entering new  chain...
What are the provinces with an area more than 1000?
SQLQuery:SELECT name
FROM province
WHERE area > 1000
SQLResult: [('Primorskiy',), ('Khabarovskiy',), ('Amurskaya',), ('Kamchatka',), ('Magadanskaya',), ('Sakhalin',), ('Khanty Mansi ao',), ('Yamalo Nenets ao',), ('Antwerpen',), ('Oost-Vlaanderen',), ('Hainaut',), ('LiÃ¨ge',), ('Limburg',), ('Luxembourg',), ('Namur',), ('West-Vlaanderen',), ('Brabant Wallon',), ('Vlaams-Brabant',), ('Luxembourg',), ('Groningen',), ('Friesland',), ('Drenthe',), ('Overijssel',), ('Flevoland',), ('Gelderland',), ('Utrecht',), ('Noord-Holland',), ('Zuid-Holland',), ('Zeeland',), ('Noord-Brabant',), ('Limburg',), ('Slovenia',), ('Croatia',), ('Federacija Bosne i Hercegovine',), ('Republika Srpska',), ('Bulgaria',), ('Alba',), ('Arad',), ('ArgeÅŸ',), ('BacÄƒu',), ('Bihor',), ('BistriÅ£a-NÄƒsÄƒud',), ('BotoÅŸani',), ('BrÄƒila',), ('BraÅŸov',), ('BuzÄƒu',), ('CÄƒlÄƒraÅŸi',), ('CaraÅŸ-Severin',), ('Cluj',), ('ConstanÅ£a',), ('Covasna'

In [34]:
execute_query("Tell me about cities of Cuba")



> Entering new  chain...
Tell me about cities of Cuba
SQLQuery:SELECT * FROM city WHERE country = 'CUB'
SQLResult: []
Answer:Final answer here: There are no cities in Cuba in the given database.
> Finished chain.


In [35]:
execute_query("city of Cuba")



> Entering new  chain...
city of Cuba
SQLQuery:SELECT name
FROM city
WHERE country = 'CUBA'
SQLResult: []
Answer:Final answer: There are no cities in Cuba in the given database.
> Finished chain.


In [36]:
execute_query("Say me about the total of city of cuba?")



> Entering new  chain...
Say me about the total of city of cuba?
SQLQuery:SELECT COUNT(*) FROM city WHERE country = 'CUBA'
SQLResult: [(0,)]
Answer:Final answer here: There are 0 cities in Cuba.
> Finished chain.


In [37]:
execute_query("Say me about the total of Cities of cuba?")



> Entering new  chain...
Say me about the total of Cities of cuba?
SQLQuery:SELECT COUNT(*) FROM city WHERE country = 'CUBA'
SQLResult: [(0,)]
Answer:Final answer: There are 0 cities in Cuba.
> Finished chain.


In [38]:
execute_query("Tell me the largest area.")



> Entering new  chain...
Tell me the largest area.
SQLQuery:SELECT MAX(area) FROM country
SQLResult: [(17075200,)]
Answer:Final answer: The largest area is 17,075,200.
> Finished chain.


In [39]:
execute_query("Areas different from 100")



> Entering new  chain...
Areas different from 100
SQLQuery:SELECT name
FROM continent
WHERE area <> 100
SQLResult: [('Europe',), ('Asia',), ('Australia/Oceania',), ('Africa',), ('North America',), ('South America',)]
Answer:Final answer here: Europe, Asia, Australia/Oceania, Africa, North America, South America
> Finished chain.


In [41]:
execute_query("Shows me cities with longitude between 0 and 6")



> Entering new  chain...
Shows me cities with longitude between 0 and 6
SQLQuery:SELECT name
FROM city
WHERE longitude BETWEEN 0 AND 6
SQLResult: [('Mons',), ('LiÃ¨ge',), ('Hasselt',), ('Arlon',), ('Namur',), ('Brugge',), ('Ostende',), ('Wavre',), ('Leuven',), ('Leeuwarden',), ('Lelystad',), ('Almere',), ('Arnhem',), ('Nijmegen',), ('Apeldoorn',), ('Utrecht',), ('Amersfoort',), ('Amsterdam',), ('Haarlem',), ('Zaanstad',), ('Hilversum',), ('Den Haag',), ('Rotterdam',), ('Dordrecht',), ('Leiden',), ('Middelburg',), ("'s-Hertogenbosch",), ('Eindhoven',), ('Tilburg',), ('Breda',), ('Maastricht',), ('Heerlen',), ('Sittard-Geleen',), ('Stavanger',), ('Bergen',), ('Bruxelles',), ('Schaerbeek',), ('Molenbeek-Saint-Jean',), ('Anderlecht',), ('Antwerpen',), ('Gent',), ('Charleroi',), ('Andorra la Vella',), ('Paris',), ('Boulogne-Billancourt',), ('Argenteuil',), ('Montreuil',), ('Saint-Denis',), ('Clermont-Ferrand',), ('Lyon',), ('Grenoble',), ('Saint-Ã‰tienne',), ('Villeurbanne',), ('Limoges',

In [42]:
execute_query("Shows me Airports with elevation more than 5000 ")



> Entering new  chain...
Shows me Airports with elevation more than 5000 
SQLQuery:SELECT * FROM airport WHERE elevation > 5000
SQLResult: []
Answer:Final answer here: There are no airports with elevation more than 5000.
> Finished chain.


In [43]:
execute_query("What language is spoken in Poland?")



> Entering new  chain...
What language is spoken in Poland?
SQLQuery:SELECT name
FROM countrylocalname
WHERE country = 'PL'
 Consulta SQL gerada, mas houve erro ao executá-la


In [44]:
execute_query("Tell me about the religion that is practiced in Haiti")



> Entering new  chain...
Tell me about the religion that is practiced in Haiti
SQLQuery:SELECT name
FROM religion
WHERE country = 'HT'
SQLResult: []
Answer:Final answer here: There is no information available about the religion practiced in Haiti in the given database.
> Finished chain.


In [45]:
execute_query("How deep is Lake Kariba?")



> Entering new  chain...
How deep is Lake Kariba?
SQLQuery:SELECT elevation
FROM lake
WHERE name = 'Kariba'
SQLResult: []
Answer:Final answer: The depth of Lake Kariba is not available in the given tables.
> Finished chain.


In [46]:
execute_query("What is the capital of Georgia?")



> Entering new  chain...
What is the capital of Georgia?
SQLQuery:SELECT capital
FROM country
WHERE name = 'Georgia'
SQLResult: [('Tbilisi',)]
Answer:The capital of Georgia is Tbilisi.
> Finished chain.


In [47]:
execute_query("What is the total of provinces of Netherlands?")



> Entering new  chain...
What is the total of provinces of Netherlands?
SQLQuery:SELECT COUNT(*) FROM province WHERE country = 'NL'
SQLResult: [(12,)]
Answer:Final answer: The total number of provinces in Netherlands is 12.
> Finished chain.


In [48]:
execute_query("total provinces Netherland")



> Entering new  chain...
total provinces Netherland
SQLQuery:SELECT COUNT(*) FROM province WHERE country = 'NL'
SQLResult: [(12,)]
Answer:Total provinces Netherland: 12
> Finished chain.


In [49]:
execute_query("total Province Netherlands")



> Entering new  chain...
total Province Netherlands
SQLQuery:SELECT COUNT(*) FROM province WHERE country = 'NL'
SQLResult: [(12,)]
Answer:Total Province Netherlands: 12
> Finished chain.


In [50]:
execute_query("What percentage of religious people are hindu in thailand?")



> Entering new  chain...
What percentage of religious people are hindu in thailand?
SQLQuery:SELECT percentage
FROM religion
WHERE country = 'TH' AND name = 'Hindu'
SQLResult: []
Answer:Final answer here: There are no records of Hindu religious people in Thailand.
> Finished chain.


In [51]:
execute_query("What are the provinces with an area greater than 1000 in Niger?")



> Entering new  chain...
What are the provinces with an area greater than 1000 in Niger?
SQLQuery:SELECT name
FROM province
WHERE country = 'NE' AND area > 1000
SQLResult: []
Answer:Question: What are the provinces with an area greater than 1000 in Niger?
SQLQuery: SELECT name
FROM province
WHERE country = 'NE' AND area > 1000
> Finished chain.


In [52]:
execute_query("The Chatanga River belongs to which country?")



> Entering new  chain...
The Chatanga River belongs to which country?
SQLQuery:SELECT country
FROM river
WHERE name = 'Chatanga'
 Consulta SQL gerada, mas houve erro ao executá-la


In [53]:
execute_query("What province is the city of Glendale in?")



> Entering new  chain...
What province is the city of Glendale in?
SQLQuery:SELECT province
FROM city
WHERE name = 'Glendale'
SQLResult: [('Arizona',), ('California',)]
Answer:Final answer: The city of Glendale is in the provinces of Arizona and California.
> Finished chain.


In [54]:
execute_query("What is the area of the city Granada in Nicaragua?")



> Entering new  chain...
What is the area of the city Granada in Nicaragua?
SQLQuery:SELECT area
FROM city
WHERE name = 'Granada' AND country = 'NI';
 Consulta SQL gerada, mas houve erro ao executá-la


In [55]:
execute_query("tell me about the Caribbean economic")



> Entering new  chain...
tell me about the Caribbean economic
SQLQuery:SELECT * FROM economy WHERE country IN (SELECT country FROM encompasses WHERE continent = 'Caribbean')
SQLResult: []
Answer:Final answer here: There is no information available about the Caribbean economy in the database.
> Finished chain.


In [56]:
execute_query("Tell me about the country with the area largest.")



> Entering new  chain...
Tell me about the country with the area largest.
SQLQuery:SELECT name
FROM country
WHERE area = (SELECT MAX(area) FROM country)
SQLResult: [('Russia',)]
Answer:The country with the largest area is Russia.
> Finished chain.


In [57]:
execute_query("What is the host province of the International Energy Agency?")



> Entering new  chain...
What is the host province of the International Energy Agency?
SQLQuery:SELECT capprov
FROM province
WHERE name = 'International Energy Agency'
SQLResult: []
Answer:Final answer: The host province of the International Energy Agency is not available in the database.
> Finished chain.


In [58]:
execute_query(" What is Mexico's independence day?")



> Entering new  chain...
 What is Mexico's independence day?
SQLQuery:SELECT independence
FROM politics
WHERE country = 'MEX';
 Consulta SQL gerada, mas houve erro ao executá-la
